In [ ]:
import pandas as pd
import numpy as np
import easygui
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import math
import re
from collections import Counter
import itertools

warnings.filterwarnings("ignore")

#nltk.download()

Usar como input desse programa um relatório com keywords, cliques, impressões e ranking position. Baseado no padrão do search console

Escolher datasets de dois pontos no tempo para gerar comparação período sobre período

Header da base de dados:
| Top queries | Clicks | Impressions | Position |

In [ ]:
#Read file and show dataframe

#filename = easygui.fileopenbox() #In case we wish to select a file from file explorer

filename = "queries FS-january2022.csv" #Current month dataset
filename_prev = "queries FS-december2021.csv"#Previous month dataset
current_month = "January 2022"
previous_month = "December 2021"

df = pd.read_csv(filename)
df_prev = pd.read_csv(filename_prev)

df.sort_values(by=['Impressions'], ascending=False, inplace=True)
df_prev.sort_values(by=['Impressions'], ascending=False, inplace=True)

# drop bad words
todrop = df[ df['Top queries'].str.contains('porno|xxx|pono|mulher nua|mulheres nuas|mulheres nua|lesbicas|shemale|xvideo') ].index
df.drop(todrop , inplace=True)

# drop bad words df_prev
todrop_prev = df_prev[ df_prev['Top queries'].str.contains('porno|xxx|pono|mulher nua|mulheres nuas|mulheres nua|lesbicas|shemale|xvideo') ].index
df_prev.drop(todrop_prev , inplace=True)

df['Top queries'] = df['Top queries'].str.replace('ps5','playstation 5')
df_prev['Top queries'] = df_prev['Top queries'].str.replace('ps5','playstation 5')
df['Top queries'] = df['Top queries'].str.replace('ps4','playstation 4')
df_prev['Top queries'] = df_prev['Top queries'].str.replace('ps4','playstation 4')

df_raw = df.copy()

In [ ]:
#Limpeza e organização do dataset

#Substituir todas as acentuações para agrupar possíveis grafias diferentes de uma mesma palavra
def accent_replacement(df):
    df['Top queries'] = df['Top queries'].str.replace('ã','a')
    df['Top queries'] = df['Top queries'].str.replace('à','a')
    df['Top queries'] = df['Top queries'].str.replace('á','a')
    df['Top queries'] = df['Top queries'].str.replace('â','a')
    df['Top queries'] = df['Top queries'].str.replace('é','e')
    df['Top queries'] = df['Top queries'].str.replace('ê','e')
    df['Top queries'] = df['Top queries'].str.replace('í','i')
    df['Top queries'] = df['Top queries'].str.replace('ó','o')
    df['Top queries'] = df['Top queries'].str.replace('õ','o')
    df['Top queries'] = df['Top queries'].str.replace('ô','o')
    df['Top queries'] = df['Top queries'].str.replace('ú','u')
    df['Top queries'] = df['Top queries'].str.replace('ç','c')
    #df['Top queries'] = df['Top queries'].str.replace('2021','') #Não considerar o termo do ano

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    WORD = re.compile(r"\w+")
    words = WORD.findall(text)
    return Counter(words)


#Identificar search terms muito semelhantes para agrupar em um único termo
def find_cosin_similarity(df):
    
    found = False
    
    kw_pairs = set(itertools.combinations(df['Top queries'].tolist(), 2))
    
    for pair in kw_pairs:
        a = get_cosine(text_to_vector(pair[0]), text_to_vector(pair[1]))
        if (a>=0.9):
            df.loc[df['Top queries'] == pair[1], 'Top queries'] = pair[0]
            found = True
    
    return found

#Pivot table to aggregate same search term
def aggr_kws(df):
    df.loc[:,'prod'] = df.loc[:,'Impressions']*df.loc[:,'Position']
    table = pd.pivot_table(df, values=['Clicks', 'Impressions', 'Position', 'prod'], index=['Top queries'], aggfunc={'Clicks': np.sum, 'Impressions': np.sum, 'Position' : np.sum, 'prod' : np.sum})
    return table.reset_index()

#Decide if the search query is a branded or unbranded term

def branded_unbranded_decision(df):
    
    #Include branded search terms here
    df['Branded'] = np.where(df['Top queries'].str.contains('amazon|kindle|echo|prime|alexa|fire stick|firetv|amaz|amzon|amazom|amaxon|amaozn|amozon|amaon|mazon|amazo|aamazon'), True, False)

    #Generate two subsets with branded and unbranded keywords
    df_branded = df[df['Branded'] == True]
    df_unbranded = df[df['Branded'] == False]
    
    #Use create_calculated_impr_colums function to add columns on dataset
    
    return df_branded, df_unbranded

#Create columns

def create_calculated_columns (df):
    '''Gets the raw dataset from GSC and create calculated columns with impressions (%)'''
    
    df.loc[:,'CTR'] = (df.loc[:,'Clicks']/df.loc[:,'Impressions'])*100  
    df['CTR'] = df['CTR'].map('{:.2f}%'.format)
    df.loc[:,'Position'] = df.loc[:,'prod']/df.loc[:,'Impressions']  
    df['Position'] = df['Position'].map('{:.2f}'.format)
    df.loc[:,'Impresions(%)'] = (df.loc[:,'Impressions']/df.loc[:,'Impressions'].sum())*100
    df.loc[:,'Cum. impr. (%)'] = df.loc[:,'Impresions(%)'].cumsum()
    return df

def sortedSentence(sentence):
      
    words = sentence.split(" ")
    words.sort()
    newSentence = " ".join(words)

    return newSentence

def average_rank(df):
    return np.average(df['Position'].astype(float))
    
def weighted_average_rank(df):
    return np.average(df['Position'].astype(float), weights = df['Impressions'].astype(float))
    
def top_kw_count(df):
    total_kws = len(df)
    top3 = df['Position'][df['Position'].astype(float) <= 3].count()/total_kws
    top5 = df['Position'][df['Position'].astype(float) <= 5].count()/total_kws
    top10 = df['Position'][df['Position'].astype(float) <= 10].count()/total_kws
    
    return top3, top5, top10;

def average_ctr(df):
    return df['Clicks'].sum()/df['Impressions'].sum()


In [ ]:
def word_frequency(df, isbranded):

    keyword_list = df['Top queries'].tolist()
    click_weight = df['Clicks'].tolist()

    tokens = []
    for t in range(len(keyword_list)):
        for i in (keyword_list[t].split()):
            if len(i)>1 and not i.isdigit(): #Considerar apenas palavras com conteúdo
                for k in range(round(click_weight[t])):
                    tokens.append(i)
                    
    ## Creating FreqDist for whole BoW, keeping the 20 most common tokens
    all_fdist = nltk.FreqDist(tokens).most_common(20)

    ## Conversion to Pandas series via Python Dictionary for easier plotting
    all_fdist = pd.Series(dict(all_fdist))
    
    ## Setting figure, ax into variables
    fig, ax = plt.subplots(figsize=(5,10))

    ## Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
    all_plot = sns.barplot(y=all_fdist.index, x=all_fdist.values, orient='h', ax=ax) 
    plt.xticks(rotation=30)
    plt.xlabel('Count of clicks', fontsize = 12)
    plt.ylabel('Search term', fontsize = 12)
    
    if isbranded == True:
        plt.title('Branded Search\n', fontsize = 15)
        ax.annotate ("'Amazon' and 'Prime' words are not considered", xy=(0,1), xycoords='axes fraction', xytext = (10,10), textcoords='offset points')
    elif isbranded == False:
        plt.title('Unbranded Search\n', fontsize = 15)
    
    return plt
    
#plt1 = word_frequency(df_branded, isbranded=True)
#plt2 = word_frequency(df_unbranded, isbranded=False)    
    

In [ ]:
def search_term_freq(df, isbranded, title):
    
    df.sort_values(by=['Clicks'], ascending=False, inplace=True)
    
    keyword_list = df['Top queries'].head(20).tolist()
    click_weight = df['Clicks'].head(20).tolist()
    
    all_fdist = pd.Series(dict(zip(keyword_list, click_weight)))
   
    ## Setting figure, ax into variables
    fig, ax = plt.subplots(figsize=(5,10))

    ## Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
    all_plot = sns.barplot(y=all_fdist.index, x=all_fdist.values, orient='h', ax=ax, color = 'gray') 
    plt.xticks(rotation=30)
    plt.xlabel('Count of clicks', fontsize = 12)
    plt.ylabel('Search term', fontsize = 12)
    
    for p in ax.patches:
        all_plot.annotate('{:,}'.format(round(p.get_width())), xy=(p.get_width(), p.get_y()+p.get_height()/2), xytext=(5, 0), textcoords='offset points', ha="right", va="center")
    
    if isbranded == True:
        plt.title('Branded Search\n', fontsize = 15)
    elif isbranded == False:
        plt.title('Unbranded Search - ' + title, fontsize = 15)
    
    return plt

In [ ]:
def word_cloud(df, stopwords):
    '''Recebe um dataframe e forma um gráfico de nuvens de palavras mais utilizadas nas strings de busca
    df deve incluir uma coluna chamada 'Top queries' ''' 
    
    # Create word list
    text = " ".join(review for review in df['Top queries'])

    # Generate image
    wordcloud = WordCloud(stopwords = stopwords, background_color="white").generate(text)

    # Display the generated image:
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

# Main

In [ ]:
#remove stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')
df['Top queries'] = df['Top queries'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df_prev['Top queries'] = df_prev['Top queries'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

accent_replacement(df)
accent_replacement(df_prev)

df['Top queries'] = df['Top queries'].apply(sortedSentence) #ordenar search terms em ordem alfabetica
df_prev['Top queries'] = df_prev['Top queries'].apply(sortedSentence)

df = aggr_kws(df)
df_prev = aggr_kws(df_prev)

#agrupar kws semelhantes
while (find_cosin_similarity(df)):
    find_cosin_similarity(df)
    df=aggr_kws(df)
    
while (find_cosin_similarity(df_prev)):
    find_cosin_similarity(df_prev)
    df_prev=aggr_kws(df_prev)

df = create_calculated_columns(aggr_kws(df))
df.drop('prod', axis=1, inplace=True)
df_branded, df_unbranded = branded_unbranded_decision(df)

df_prev = create_calculated_columns(aggr_kws(df_prev))
df_prev.drop(['prod'], axis = 1, inplace=True)
df_branded_prev, df_unbranded_prev = branded_unbranded_decision(df_prev)


In [ ]:
#Unbranded dataset visualization
pd.set_option('display.max_rows', None)
df_unbranded.sort_values(by=['Clicks'], ascending=False, inplace=True)
df_unbranded

In [ ]:
#Branded dataset visualization
pd.set_option('display.max_rows', None)
df_branded.sort_values(by=['Clicks'], ascending=False, inplace=True)
df_branded

In [ ]:
#Plot Search term frequency
plt1 = search_term_freq(df_unbranded, False, current_month)
plt2 = search_term_freq(df_unbranded_prev, False, previous_month)     

In [ ]:
#Print Ranking statistics

nl = '\n'

print(f"Average rank: {average_rank(df_unbranded):.2f}{nl}Weighted average rank: {weighted_average_rank(df_unbranded):.2f}{nl}Average CTR: {average_ctr(df_unbranded):.2%}{nl}Top 3 kw kw share: {top_kw_count(df_unbranded)[0]:.2%}")
print(f"Top 5 kw kw share: {top_kw_count(df_unbranded)[1]:.2%}{nl}Top 10 kw kw share: {top_kw_count(df_unbranded)[2]:.2%}")

In [ ]:
#MoM keywords comparisom

df_unbranded.sort_values(by=['Clicks'], ascending=False, inplace=True)
df_unbranded_prev.sort_values(by=['Clicks'], ascending=False, inplace=True)

kws_current_month = set(df_unbranded['Top queries'].head(20).apply(lambda x: sortedSentence(x)))
kws_previous_month = set(df_unbranded_prev['Top queries'].head(20).apply(lambda x: sortedSentence(x)))


print("Out of Top-20: ")
print(kws_previous_month - kws_current_month)

print("\nInto of Top-20: ")
print(kws_current_month - kws_previous_month)



In [ ]:
word_cloud(df_branded, stopwords)
word_cloud(df_unbranded, stopwords)

In [ ]:
#Combining data into one single dataframe for MoM comparisson

df_comb = df_unbranded.join(df_unbranded_prev.set_index('Top queries'), on = 'Top queries', how='outer', lsuffix=' current', rsuffix=' previous', sort=False)
df_comb

df_comb['Clicks variation'] =  df_comb['Clicks current']/df_comb['Clicks previous'] - 1
df_comb['Ranking variation'] =  df_comb['Position previous'].astype(float) - df_comb['Position current'].astype(float)


In [ ]:
topx = 5 #How many top keywords to display

#TOP GROWING KEYWORDS
top_growing_kws = df_comb[(df_comb['Clicks current'] >= 500) | (df_comb['Clicks previous'] >= 500)]
top_growing_kws.sort_values(by=['Clicks variation'], ascending=False, inplace=True)
top_growing_kws.index = pd.RangeIndex(start=1, stop=len(top_growing_kws)+1, step=1)
print("\nTOP GROWING KEYWORDS")
top_growing_kws = top_growing_kws[['Top queries', 'Clicks current', 'Clicks previous', 'Clicks variation']].head(topx)
display(top_growing_kws)

#TOP DESCENDING KEYWORDS
top_desc_kws = df_comb[(df_comb['Clicks current'] >= 500) | (df_comb['Clicks previous'] >= 500)]
top_desc_kws.sort_values(by=['Clicks variation'], ascending=True, inplace=True)
top_desc_kws.index = pd.RangeIndex(start=1, stop=len(top_desc_kws)+1, step=1)
print("\nTOP DESCENDING KEYWORDS")
top_desc_kws = top_desc_kws[['Top queries', 'Clicks current', 'Clicks previous', 'Clicks variation']].head(topx)
display(top_desc_kws)

#TOP RANK GROWTH
top_rank_growth = df_comb[(df_comb['Clicks current'] >= 500) | (df_comb['Clicks previous'] >= 500)]
top_rank_growth.sort_values(by=['Ranking variation'], ascending=False, inplace=True)
top_rank_growth.index = pd.RangeIndex(start=1, stop=len(top_rank_growth)+1, step=1)
print("\nTOP RANK GROWTH")
top_rank_growth = top_rank_growth[['Top queries', 'Position current', 'Position previous', 'Ranking variation']].head(topx)
display(top_rank_growth)

#TOP RANK DECLINE
top_rank_decl = df_comb[(df_comb['Clicks current'] >= 500) | (df_comb['Clicks previous'] >= 500)]
top_rank_decl.sort_values(by=['Ranking variation'], ascending=True, inplace=True)
top_rank_decl.index = pd.RangeIndex(start=1, stop=len(top_rank_decl)+1, step=1)
print("\nTOP RANK DECLINE")
top_rank_decl = top_rank_decl[['Top queries', 'Position current', 'Position previous', 'Ranking variation']].head(topx)
display(top_rank_decl)

#TOP NEW KEYWORDS
top_new_kw = df_comb[(df_comb['Clicks current'] > 0) & (df_comb['Clicks previous'].isnull())]
top_new_kw.sort_values(by=['Clicks current'], ascending=False, inplace=True)
top_new_kw.index = pd.RangeIndex(start=1, stop=len(top_new_kw)+1, step=1)
print("\nTOP NEW KEYWORDS")
top_new_kw = top_new_kw[['Top queries', 'Position current', 'Clicks current']].head(topx)
display(top_new_kw)

#HIGH CTR KEYWORDS
high_ctr_kws = df_comb.copy()
high_ctr_kws.sort_values(by=['CTR current'], ascending=False, inplace=True)
high_ctr_kws.index = pd.RangeIndex(start=1, stop=len(high_ctr_kws)+1, step=1)
print("\nHIGH CTR KEYWORDS")
high_ctr_kws = high_ctr_kws[['Top queries', 'Clicks current', 'Impressions current', 'CTR current']].head(2*topx)
display(high_ctr_kws)

#LOW CTR KEYWORDS
#low_ctr_kws = df_comb[(df_comb['Clicks current'] >= 500) | (df_comb['Clicks previous'] >= 500)]
low_ctr_kws = df_comb.copy()
low_ctr_kws.sort_values(by=['CTR current'], ascending=True, inplace=True)
low_ctr_kws.index = pd.RangeIndex(start=1, stop=len(low_ctr_kws)+1, step=1)
print("\nLOW CTR KEYWORDS")
low_ctr_kws = low_ctr_kws[['Top queries', 'Clicks current', 'Impressions current', 'CTR current']].head(2*topx)
display(low_ctr_kws)

titles = ["Top growing keywords", "Top descending keywords", "Top rank growth", "Top ranking decline", "Top new keywords", "High CTR keywords", "Low CTR keywords"]
tables = [top_growing_kws, top_desc_kws, top_rank_growth, top_rank_decl, top_new_kw, high_ctr_kws, low_ctr_kws]

#Write tables on Excel
writer = pd.ExcelWriter('kws_analysis.xlsx', engine='xlsxwriter')
row = 0
for title, table in zip(titles, tables):
    table.to_excel(writer, sheet_name='Sheet1', startrow=row+1)  
    worksheet = writer.sheets['Sheet1']
    worksheet.write(row, 0, title)
    row += len(table) + 3
writer.save()

In [ ]:
def buscar_querie(df):
    string = str(input("Buscar palavra chave:"))
    recorte = df[df['Top queries'].str.contains(string)].iloc[:,0:4]
    print("Total de cliques: ", sum(recorte['Clicks']) )
    return recorte
    
buscar_querie(df_raw)
